In [1]:

import numpy as np
import warnings
from tensorflow.keras.callbacks import EarlyStopping
warnings.filterwarnings("ignore")
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow_hub as hub
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv
GPU is available


2022-05-06 20:32:12.343195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 20:32:12.454916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 20:32:12.455734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
df = pd.read_csv('../input/200k-short-texts-for-humor-detection/dataset.csv')
df['humor']=df['humor'].map({True:1,False:0})
df.head(2)

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",0
1,Watch: darvish gave hitter whiplash with slow ...,0


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['humor'], test_size=0.3)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_val=np.array(X_test)
y_val=np.array(y_test)
X_test=X_val[:1000]
y_test=y_val[:1000]
X_val=X_val[1000:]
y_val=y_val[1000:]

In [4]:
X_test[:5]

array(["I know a girl who is completely obsessed with mexican men. she's a pedrophile.",
       "Health care is 'complicated' - here's one way to fix it",
       'What happens in vegas stays on facebook, instagram, youtube, vine, and medical records.',
       "What's invisible and smells like carrots? bunny farts.",
       "My attitude changes in 5 seconds flat. from sweetheart to bitch. so i suggest you don't test that."],
      dtype=object)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
for example, label in validation_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'How many feminists does it take to change a light bulb? just one. and that is ***not funny***.'
label:  1


2022-05-06 20:32:13.127520: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 20:32:13.128006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 20:32:13.128875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 20:32:13.129570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [7]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

2022-05-06 20:32:15.356854: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [8]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)#, activation='sigmoid'
])
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True, True]


In [9]:
# predict on a sample text without padding.

sample_text = ('What do you call a bee that can’t make up its mind? A maybe.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

2022-05-06 20:32:27.997795: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


[-0.00421236]


In [10]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[-0.00421236]


In [11]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [12]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model_rnn",
                                            save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_dataset, epochs=8,
                    validation_data=validation_dataset,
                    validation_steps=30,callbacks=[stop_early,checkpoint])

Epoch 1/8
2188/2188 [==============================] - 66s 24ms/step - loss: 0.2397 - accuracy: 0.8903 - val_loss: 0.1800 - val_accuracy: 0.9276


2022-05-06 20:33:50.766440: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/8
2188/2188 [==============================] - 51s 24ms/step - loss: 0.1805 - accuracy: 0.9271 - val_loss: 0.1760 - val_accuracy: 0.9219
Epoch 3/8
2188/2188 [==============================] - 53s 24ms/step - loss: 0.1738 - accuracy: 0.9298 - val_loss: 0.1698 - val_accuracy: 0.9297
Epoch 4/8
2188/2188 [==============================] - 53s 24ms/step - loss: 0.1700 - accuracy: 0.9312 - val_loss: 0.1679 - val_accuracy: 0.9286
Epoch 5/8
2188/2188 [==============================] - 54s 25ms/step - loss: 0.1669 - accuracy: 0.9323 - val_loss: 0.1675 - val_accuracy: 0.9344
Epoch 6/8
2188/2188 [==============================] - 53s 24ms/step - loss: 0.1641 - accuracy: 0.9337 - val_loss: 0.1658 - val_accuracy: 0.9276
Epoch 7/8
2188/2188 [==============================] - 51s 23ms/step - loss: 0.1614 - accuracy: 0.9349 - val_loss: 0.1613 - val_accuracy: 0.9354
Epoch 8/8
2188/2188 [==============================] - 49s 22ms/step - loss: 0.1586 - accuracy: 0.9356 - val_loss: 0.1619 - val_ac